# Using the Unsafe Foods Database

## Understanding the Database

![alt text]({{ site.url }}/assets/images/UnsafeFoodsDatabase.png "Unsafe Foods DB Schema")

The Unsafe Foods Database is a relational Postgres database that facilitates the querying and analysis of Amazon review data with respect to historical FDA recall data. We gathered the Amazon Review data from a [historical dataset](http://jmcauley.ucsd.edu/data/amazon/) that was published by Julian McAuley at UCSD. This data contains approximately 1.8 million individual reviews that include the reviewer ID, date and time of the review, the text, summary, rating, and product metadata.  

## Connecting to the Database in Python

In python, you need to download a module to connect with the PostGres database. My preferred module is psycopg2. In order to install this package successfully, you must install its dependencies first using the following commands in bash: 

```bash
sudo apt-get install gcc
sudo apt-get install python-setuptools
sudo easy_install psycopg2
```

After installing, you can invoke the module in your Python script and connect to the database using the following command and parameters:

```python
import psycopg2

#Connect to database
conn = psycopg2.connect(database = <Database-Name>, user = <your-user-name>, password = <your-password>, host = <your-host-name>, port = <your port number, usually 5432>)

print("Opened database successfully")
```

## Querying the Database

Using the above connection info, this section will connect to the database and execute some queries to illustrate the database's capabilities.

In [2]:
import psycopg2

#We also want to import pandas and numpy to work with the data we have fetched
import pandas as pd
import numpy as np

#Connect to database; input depends on your settings
conn = psycopg2.connect(database=<db_name>, user=<user_name>, password=<password>, host=<host_name>, port=<port_name>)

print("Opened database successfully")

Opened database successfully


First, let's go ahead and look at some of the metadata that we are working with. How many products are there per category?

In [3]:
'''
You need to set up a cursor before you start executing queries.
One way to look at it is that your connection, 'conn', is your ticket to the database,
while your cursor, cur, will be your shopping cart.
'''
cur = conn.cursor()

In [4]:
#execute SQL query
cur.execute('SELECT c.category_name, count(*) as NumProducts from\
                Category c Join CategoryAssignment ca on c.category_id = ca.category_id\
                JOIN Product p on ca.product_id = p.product_id\
                group by c.category_name order by NumProducts DESC;')

#fetch table from the cursor
category_breakdown = pd.DataFrame(cur.fetchall())

In [5]:
category_breakdown

,0,1
0,Grocery & Gourmet Food,171760
1,Beverages,3925
2,Cooking & Baking,2434
3,Tea,1791
4,Chocolate,1043
5,Snack Foods,983
6,Coffee,949
7,Candy & Chocolate,927
8,Tea Samplers,763
9,Herbal,583


What about the number of reviews per category?

In [6]:
#execute SQL query
cur.execute('SELECT c.category_name, count(*) as NumReviews from\
                Category c Join CategoryAssignment ca on c.category_id = ca.category_id\
                JOIN Review r on ca.product_id = r.product_id\
                group by c.category_name order by NumReviews DESC;')

#fetch table from the cursor
category_breakdown_reviews = pd.DataFrame(cur.fetchall())

In [7]:
category_breakdown_reviews

,0,1
0,Grocery & Gourmet Food,1077410
1,Beverages,23570
2,Cooking & Baking,20297
3,Tea,9737
4,Sugar,5793
5,Vinegars,4792
6,Oils,4792
7,Chocolate,4482
8,Coffee,4431
9,Candy & Chocolate,3981


How about we start looking at recalled products? Maybe a certain category gets a lot of recalls? 

In [8]:
#execute SQL query
cur.execute('SELECT c.category_name, count(*) as NumRecalls from\
                Category c Join CategoryAssignment ca on c.category_id = ca.category_id\
                JOIN recalledproduct rp on ca.product_id = rp.product_id\
                where ca.product_id in (select product_id from recalledproduct)\
                group by c.category_name order by NumRecalls DESC;')

#fetch table from the cursor
category_breakdown_recalls = pd.DataFrame(cur.fetchall())

In [9]:
category_breakdown_recalls

,0,1
0,Grocery & Gourmet Food,158
1,Bars,6
2,Nut,5
3,Snack Foods,4
4,Single Herbs & Spices,3
5,Butter,3
6,Energy & Nutritional,3
7,Cereal,3
8,Candy & Chocolate,3
9,Breakfast Foods,3


All of our products are in the Grocery and Gourmet Food category, but it looks like Canned Dry & Packaged Foods might be the biggest headache for Amazon.</br>
What about brands? What is the range of number of recalls that a single brand endured within the time frame of our recall data set?

In [10]:
#execute SQL query
cur.execute('SELECT b.brand_id, count(*) as NumRecalls from\
                Brand b Join Product p on b.brand_id = p.brand_id\
                join recalledproduct rp on p.product_id = rp.product_id\
                group by b.brand_id order by NumRecalls DESC;')

#fetch table from the cursor
brand_breakdown = pd.DataFrame(cur.fetchall())

In [11]:
brand_breakdown

,0,1
0,5158,13
1,7169,8
2,2650,8
3,9864,8
4,6327,5
5,1154,5
6,8812,4
7,7259,4
8,8705,3
9,3321,3


Don't forget to close your connection!

In [12]:
conn.close()